<a href="https://colab.research.google.com/github/adeakinwe/computer_vision/blob/main/hyper_parameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import wandb
from wandb.integration.keras import WandbMetricsLogger
import tensorflow as tf

In [8]:
sweep_config = {
    'method': 'grid',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'learning_rate': {
            'values': [0.001, 0.0001]
        },
        'batch_size': {
            'values': [8, 16]
        },
        'hidden_nodes': {
            'values': [128, 64],
        },
        'img_size': {
            'values': [16, 224]
        },
        'epochs': {
            'values': [5, 10]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="5-flowers-sweep")

Create sweep with ID: qgsqn7e0
Sweep URL: https://wandb.ai/adeakinwe-richkinwe-io/5-flowers-sweep/sweeps/qgsqn7e0


In [10]:
from tensorflow import keras
import matplotlib.pyplot as plt

In [11]:
def train():
  with wandb.init() as run:
    config = wandb.config

  IMG_HEIGHT = config.img_size
  IMG_WIDTH = config.img_size
  IMG_CHANNELS = 3

  CLASS_NAMES = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]

  def read_and_decode(filename, resize_dims):
    # 1. Read the raw file
    img_bytes = tf.io.read_file(filename)
    # 2. Decode image data
    img = tf.image.decode_jpeg(img_bytes, channels=IMG_CHANNELS)
    # 3. Convert pixel values to floats in [0, 1]
    img = tf.image.convert_image_dtype(img, tf.float32)
    # 4. Resize the image to match desired dimensions
    img = tf.image.resize(img, resize_dims)
    return img

  def parse_csvline(csv_line):
    # record_defaults specify the data types for each column
    record_default = ["", ""]
    filename, label_string = tf.io.decode_csv(csv_line, record_default)

    # Load the image
    img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])

    # Convert label string to integer based on the CLASS_NAMES index
    label = tf.argmax(tf.math.equal(CLASS_NAMES, label_string))
    return img, label

    # Define datasets
    train_dataset = (
        tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/train_set.csv")
        .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(config.batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    eval_dataset = (
        tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/eval_set.csv")
        .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(config.batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    model = keras.Sequential([
    keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
    keras.layers.Dense(config.hidden_nodes, activation="relu"),
    keras.layers.Dense(len(CLASS_NAMES), activation="softmax")
   ])

    model.compile(
    optimizer="adam",
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"]
   )

    model.fit(
    train_dataset,
    validation_data=eval_dataset,
    epochs=config.epochs,
    callbacks=[
      WandbMetricsLogger(log_freq=5)
    ]
   )

In [12]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: w8aqepg1 with config:
wandb: 	batch_size: 8
wandb: 	epochs: 5
wandb: 	hidden_nodes: 128
wandb: 	img_size: 16
wandb: 	learning_rate: 0.001


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 5wgvewre with config:
wandb: 	batch_size: 8
wandb: 	epochs: 5
wandb: 	hidden_nodes: 128
wandb: 	img_size: 16
wandb: 	learning_rate: 0.0001


wandb: Agent Starting Run: rw30r2ug with config:
wandb: 	batch_size: 8
wandb: 	epochs: 5
wandb: 	hidden_nodes: 128
wandb: 	img_size: 224
wandb: 	learning_rate: 0.001


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: lyssfbyx with config:
wandb: 	batch_size: 8
wandb: 	epochs: 5
wandb: 	hidden_nodes: 128
wandb: 	img_size: 224
wandb: 	learning_rate: 0.0001


wandb: Agent Starting Run: fap2hf1l with config:
wandb: 	batch_size: 8
wandb: 	epochs: 5
wandb: 	hidden_nodes: 64
wandb: 	img_size: 16
wandb: 	learning_rate: 0.001


...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: lvjtxasw with config:
wandb: 	batch_size: 16
wandb: 	epochs: 10
wandb: 	hidden_nodes: 64
wandb: 	img_size: 16
wandb: 	learning_rate: 0.001


wandb: Agent Starting Run: 2w9gwin2 with config:
wandb: 	batch_size: 16
wandb: 	epochs: 10
wandb: 	hidden_nodes: 64
wandb: 	img_size: 16
wandb: 	learning_rate: 0.0001


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 9qek7n0q with config:
wandb: 	batch_size: 16
wandb: 	epochs: 10
wandb: 	hidden_nodes: 64
wandb: 	img_size: 224
wandb: 	learning_rate: 0.001


wandb: Agent Starting Run: bya9ogl3 with config:
wandb: 	batch_size: 16
wandb: 	epochs: 10
wandb: 	hidden_nodes: 64
wandb: 	img_size: 224
wandb: 	learning_rate: 0.0001


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
